In [2]:
!pip install -q --upgrade pip
!pip install -q requests bs4 

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


def getLastModifiedTime(url):
    response = requests.head(url)
    if 'Last-Modified' in response.headers:
        return response.headers['Last-Modified']
    else:
        return None


def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return parent


def is_within_hour(last_modified_time, hr):
    # Convert Last-Modified time to datetime object
    last_modified_datetime = datetime.strptime(last_modified_time, '%a, %d %b %Y %H:%M:%S %Z')

    # Calculate the time difference
    time_difference = datetime.utcnow() - last_modified_datetime

    # Check if the file was modified within the last hour
    return time_difference < timedelta(hours=hr)

In [8]:
pdf_folder_path = "infohub"
url = "http://http-port-fileserver.apps.privateailab.acic.accenture/pdf/demo/demo/"
files_to_embed = []
updated_file_lookup_hours = 1

In [9]:
result = get_url_paths(url, "pdf")
modified_pdf_list = '/data/modified_pdf_list.txt'

In [10]:
for pdf_url in result:
    last_modified_time = getLastModifiedTime(pdf_url)
    if last_modified_time is not None and is_within_hour(last_modified_time, updated_file_lookup_hours):
        filename = pdf_url.split('/')[-1]
        files_to_embed.append(filename)
        print(filename + " has been stored in" + modified_pdf_list)

with open(modified_pdf_list, 'w') as file:
    for item in files_to_embed:
        file.write("%s\n" % item)


FileNotFoundError: [Errno 2] No such file or directory: '/data/modified_pdf_list.txt'